<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">


# Derivatives Analytics with Python
**&mdash;Part IIIb: Market-Based Valuation (Ch.&nbsp;11&ndash;13)**

&copy; Dr. Yves J. Hilpisch | The Python Quants

<a href="https://tpq.io" target="_blank">tpq.io</a> | <a href="https://linktr.ee/dyjh" target="_blank">linktr.ee/dyjh</a>

<img src="https://hilpisch.com/dawp_cover_small.png" width=30% align=left>


## Part IIIb &mdash; Market-Based Valuation

### Chapter 11 &mdash; Model Calibration

This notebook implements the calibration routines for the Bakshi&mdash;Cao&mdash;Chen (BCC)
model used in Part IIIb.

- Load EURO STOXX 50 option data and construct time-to-maturity.
- Specify short-rate, stochastic volatility, and jump parameters.
- Calibrate a simplified BCC parameter set to market option prices.
- Store calibrated parameters for later valuation and hedging notebooks.

Reference scripts from the original companion repository are located under
`x_store/dawp/python36/11_cal`.


### 1. Environment and figure configuration

We start by importing the core libraries for this class, printing basic
version information, and configuring the figure export helper used to
generate PDFs for the slide deck.


In [ ]:
import sys  # access basic runtime information
from pathlib import Path  # path handling for data and figure export

import math  # elementary math functions

import numpy as np  # numerical arrays
import pandas as pd  # tabular data handling
import matplotlib as mpl  # matplotlib configuration
import matplotlib.pyplot as plt  # plotting

import scipy.optimize as sop  # numerical optimization

np.set_printoptions(precision=6, suppress=True)  # compact numeric output

print(sys.version.split()[0])  # Python version string
print("NumPy:", np.__version__)  # NumPy version string
print("pandas:", pd.__version__)  # pandas version string
print("matplotlib:", mpl.__version__)  # matplotlib version string

FIG_SAVE = True  # set to True to export figures as PDFs
FIG_DIR = Path("../figures")  # figure output directory
FIG_DPI = 300  # target resolution for exported figures
FIG_DISPLAY = "svg"  # inline display format: "svg" or "png"

plt.style.use("seaborn-v0_8")  # readable plotting defaults
mpl.rcParams["figure.figsize"] = (8.0, 4.5)  # consistent figure size
mpl.rcParams["axes.grid"] = True  # show a grid for readability
mpl.rcParams["savefig.dpi"] = FIG_DPI  # default export resolution

try:
    from matplotlib_inline.backend_inline import set_matplotlib_formats
    set_matplotlib_formats(FIG_DISPLAY)  # configure inline plot rendering
except Exception:
    pass

if FIG_DISPLAY == "png":
    mpl.rcParams["figure.dpi"] = FIG_DPI  # high-resolution inline plots


def maybe_save(fig, filename):
    # Optionally saves a Matplotlib figure as a PDF file.
    if not FIG_SAVE:
        return
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    path = FIG_DIR / f"{filename}.pdf"
    fig.savefig(path, format="pdf", dpi=FIG_DPI)
    print(f"saved: {path}")


CODE_DIR = Path("..") / "code"  # companion module directory
sys.path.insert(0, str(CODE_DIR.resolve()))  # prefer local modules


### 2. EURO STOXX 50 option data

We now load the EURO STOXX 50 option data used in Chapters 8&ndash;13 and
construct a near-the-money subset that serves as the calibration
universe for the BCC model.


In [ ]:
from dawp_pIIIb_ch11_bcc_calibration import load_option_data

data = load_option_data()  # EURO STOXX 50 option quotes
S0 = 3225.93  # EURO STOXX 50 index level (30.09.2014)

tol = 0.02  # relative moneyness window
options = data[(np.abs(data["Strike"] - S0) / S0) < tol].copy()
  # near-the-money option selection

options[["Strike", "T", "Call"]].head(10)


### 3. BCC parameter calibration

Finally we either load a stored BCC parameter set or (optionally) run
the full calibration routine. The calibrated parameters are reused in
the valuation and hedging notebooks of Chapters 12 and 13.


In [ ]:
import importlib  # module reloading for development
import dawp_pIIIb_ch11_bcc_calibration as bcc_cal  # calibration helpers

bcc_cal = importlib.reload(bcc_cal)  # ensure the latest version is active
from dawp_pIIIb_ch11_bcc_calibration import calibrate_bcc_full, load_bcc_parameters

params = load_bcc_parameters()  # calibrated or fallback parameter set
print('Initial/fallback parameters:')
for key, val in params.items():
    print(f'  {key}: {val:.4f}')

params, err = calibrate_bcc_full(
    options,
    S0=S0,
    r0=0.0005,
    verbose=True,
)
print('\nCalibrated parameters (RMSE in index points):', f'{err:.4f}')
for key, val in params.items():
    print(f'  {key}: {val:.4f}')


### 4. Calibration fit visualization

We now visualize the calibration result by comparing market and model prices
across strikes for each maturity.


In [ ]:
from dawp_pIIIa_ch09_bcc_option_valuation import bcc_call_value_int, cir_implied_short_rate

maturities = sorted(options['T'].unique())  # time-to-maturity values in years

fig, axes = plt.subplots(len(maturities), 1, sharex=True,
                         figsize=(8.0, 2.8 * len(maturities)))

if len(maturities) == 1:
    axes = [axes]

for ax, T in zip(axes, maturities):
    sub = options[options['T'] == T].sort_values('Strike')
    strikes = sub['Strike'].to_numpy()
    calls = sub['Call'].to_numpy()

    r_eff = cir_implied_short_rate(
        params['r0'],
        params['kappa_r'],
        params['theta_r'],
        params['sigma_r'],
        T,
    )

    model = [
        bcc_call_value_int(
            params['S0'],
            float(K),
            float(T),
            float(r_eff),
            params['kappa_v'],
            params['theta_v'],
            params['sigma_v'],
            params['rho'],
            params['v0'],
            params['lamb'],
            params['mu_J'],
            params['delta'],
        )
        for K in strikes
    ]

    ax.plot(strikes, calls, 'b-', label='market')
    ax.plot(strikes, model, 'ro', ms=3.5, label='model')
    ax.set_ylabel('call value')
    ax.set_title(f'maturity T = {T:.2f} years')

axes[-1].set_xlabel('strike K')
axes[-1].legend(loc=0)
fig.tight_layout()
maybe_save(fig, 'dawp_pIIIb_fig01_bcc_calibration_fit')


<img src="https://hilpisch.com/tpq_logo_bic.png"
     width="30%"
     align="right"
     style="border-radius: 8px;">
